## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-16-01-15-51 +0000,nyt,State and Federal Lawmakers Want Data Centers ...,https://www.nytimes.com/2026/01/15/business/en...
1,2026-01-16-01-15-10 +0000,nypost,Hochul pledged to save NYers a bundle of money...,https://nypost.com/2026/01/15/us-news/hochul-p...
2,2026-01-16-01-14-13 +0000,nyt,Couple Says ICE Agents Gassed Them as They Dro...,https://www.nytimes.com/2026/01/15/us/minneapo...
3,2026-01-16-01-13-17 +0000,nyt,Venezuela’s Interim Leader Delivers State of t...,https://www.nytimes.com/2026/01/15/world/ameri...
4,2026-01-16-01-12-29 +0000,nyt,Former Senator Kyrsten Sinema Accused of Affai...,https://www.nytimes.com/2026/01/15/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2392/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
57,trump,62
48,iran,30
19,ice,21
91,new,19
25,venezuela,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
114,2026-01-15-21-46-24 +0000,nypost,Venezuelan opposition leader María Corina Mach...,https://nypost.com/2026/01/15/us-news/venezuel...,150
151,2026-01-15-20-04-06 +0000,bbc,Trump threatens to invoke Insurrection Act to ...,https://www.bbc.com/news/articles/cwy15596p98o...,143
39,2026-01-15-23-31-25 +0000,latimes,Venezuela opposition leader Machado says she '...,https://www.latimes.com/politics/story/2026-01...,138
14,2026-01-16-00-46-28 +0000,wapo,Trump threatens Insurrection Act deployment to...,https://www.washingtonpost.com/nation/2026/01/...,137
274,2026-01-15-13-36-16 +0000,nypost,Trump threatens to invoke Insurrection Act if ...,https://nypost.com/2026/01/15/us-news/trump-th...,136


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
114,150,2026-01-15-21-46-24 +0000,nypost,Venezuelan opposition leader María Corina Mach...,https://nypost.com/2026/01/15/us-news/venezuel...
151,81,2026-01-15-20-04-06 +0000,bbc,Trump threatens to invoke Insurrection Act to ...,https://www.bbc.com/news/articles/cwy15596p98o...
167,72,2026-01-15-19-20-00 +0000,wsj,"In an interview, the Philly Fed’s new presiden...",https://www.wsj.com/economy/central-banking/an...
188,62,2026-01-15-18-04-50 +0000,nypost,White House warns Iran will face ‘grave conseq...,https://nypost.com/2026/01/15/us-news/sen-lind...
372,45,2026-01-15-02-13-51 +0000,cbc,Video shows woman dragged from car by ICE agen...,https://www.cbc.ca/news/world/ice-woman-car-mi...
149,43,2026-01-15-20-09-34 +0000,bbc,US forces seize a sixth Venezuela-linked oil t...,https://www.bbc.com/news/articles/cy4qdnj5vl9o...
57,41,2026-01-15-23-01-40 +0000,bbc,South Korea's impeached leader faces first of ...,https://www.bbc.com/news/articles/cy9y039ygdro...
223,41,2026-01-15-16-34-29 +0000,nypost,Judge rejects Paramount Skydance request to sp...,https://nypost.com/2026/01/15/media/judge-reje...
246,40,2026-01-15-15-06-43 +0000,nypost,2 dead in Thailand after crane crashes onto ro...,https://nypost.com/2026/01/15/world-news/two-d...
44,36,2026-01-15-23-25-28 +0000,nypost,Accused killer Paul Caneiro was ‘very quiet’ w...,https://nypost.com/2026/01/15/us-news/accused-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
